# BrainWeb: 20 Anatomical Models of 20 Normal Brains
This notebook downloads the data from the brainweb anatomical models. Each file normally has to be downloaded automatically, this script automates the process. The download takes quite a while and will occupy about 1.4 Gb. The data is downloaded as raw unsigned integers and saved into a nifti file.

Use this script at your own risk. 

Don't forget to cite these references if you use this data in any publication

- B. Aubert-Broche, D.L. Collins, A.C. Evans: "A new improved version of the realistic digital brain phantom"
NeuroImage, in review - 2006.
- B. Aubert-Broche, M. Griffin, G.B. Pike, A.C. Evans and D.L. Collins: "20 new digital brain phantoms for creation of validation image data bases"
IEEE TMI, in review - 2006

In [2]:
import numpy as np
import requests
import nibabel as nib
import os

In [16]:
# Base download params
def make_download_params(sub, email, name, institution, data_format='raw_short', zip_value='none'):
    download_params = {
        'format_value':data_format,
        'zip_value':zip_value,
        'who_name':name,
        'who_institution':institution,
        'who_email':email,
        'download_for_real': '[Start download!]', 
        'do_download_alias':sub,
        }

    return download_params

# Brainweb1 is the normal brainweb dataset with 20 subjects
datashape = [362, 434, 362]
offset = [-72.25, -126.25, -90.25]
spacing = [0.5, 0.5, 0.5, 0]
affine = np.diag(spacing)
affine[:3,3] = offset
brainweb1 = {'url':'https://brainweb.bic.mni.mcgill.ca/cgi/brainweb1',
            'affine': affine,
            'datashape':datashape}

# Brainweb2 is the brainweb dataset with MS lesion
datashape = [181, 217, 181]
offset = [-72, -126, -90]
spacing = [1, 1, 1, 0]
affine = np.diag(spacing)
affine[:3,3] = offset
brainweb2 = {'url':'https://brainweb.bic.mni.mcgill.ca/cgi/brainweb2',
            'affine': affine,
            'datashape':datashape}


def download_file(download_params, url, affine, datashape, folder):
    """Downloads a single file from brainweb and saves it as a nifti file.

    Args:
        download_params (dict): Download parameters for the brainweb website.
        url (str): Base download url.
        affine (array): 4x4 array affine matrix.
        folder (str): Output folder.
    """
    fname = download_params['do_download_alias']
    nii_fname = f'{folder}/{fname}.nii.gz'

    if os.path.exists(nii_fname):
        print(f'File {nii_fname} already exists. Skipping download.')
        return
        
    print(f'Starting download of {fname}...', end=' ')
    response = requests.get(url, params=download_params)
    print(response.url)
    print('[Done]')
    print('Saving data...', end=' ')

    raw_data = np.frombuffer(response.content, dtype=np.short)
    raw_data = np.rot90(raw_data.reshape(datashape), axes=(0,2))

    nii = nib.Nifti1Image(raw_data,affine)
    nib.save(nii, nii_fname)
    print(f"Saved data to {nii_fname}.nii.gz")


def download_20sub_brainweb(email, name, institution, folder):
    tissues = ['bck', 'csf', 'gry', 'wht', 'fat', 'mus', 'm-s', 'skl', 'ves', 'fat2', 'dura', 'mrw']
    subs = ['04','05','06','18','20','38','41','42','43','44','45','46','47','48','49','50','51','52','53','54']

    for tis in tissues:
        for sub in subs:
            params = make_download_params(f"subject{sub}", email, name, institution)
            download_file(params, brainweb1['url'], brainweb1['affine'], brainweb1['datashape'], folder)

def download_MS_brainweb(email, name, institution, folder):
    tissues = ['bck', 'csf', 'gry', 'wht', 'fat', 'm-s', 'skn', 'skl', 'gli', 'mit','wml']
    for sub in [1,2,3]:
        for tis in tissues:
            sub_name=f'phantom_1.0mm_msles{sub}_{tis}'
            params = make_download_params(sub_name, email, name, institution)
            download_file(params, brainweb2['url'], brainweb2['affine'], brainweb2['datashape'], folder)
    

In [17]:
name = 'Emil Ljungberg'
institution = 'Lund University'
email = 'emil.ljungberg@med.lu.se'
folder = 'brainweb_data'
# download_20sub_brainweb(email, name, institution, folder)
MS_folder = 'brainweb_MS_data'
download_MS_brainweb(email, name, institution, MS_folder)

Starting download of phantom_1.0mm_msles1_bck... https://brainweb.bic.mni.mcgill.ca/cgi/brainweb2?format_value=raw_short&zip_value=none&who_name=Emil+Ljungberg&who_institution=Lund+University&who_email=emil.ljungberg%40med.lu.se&download_for_real=%5BStart+download%21%5D&do_download_alias=phantom_1.0mm_msles1_bck
[Done]
Saving data... Saved data to brainweb_MS_data/phantom_1.0mm_msles1_bck.nii.gz.nii.gz
Starting download of phantom_1.0mm_msles1_csf... https://brainweb.bic.mni.mcgill.ca/cgi/brainweb2?format_value=raw_short&zip_value=none&who_name=Emil+Ljungberg&who_institution=Lund+University&who_email=emil.ljungberg%40med.lu.se&download_for_real=%5BStart+download%21%5D&do_download_alias=phantom_1.0mm_msles1_csf
[Done]
Saving data... Saved data to brainweb_MS_data/phantom_1.0mm_msles1_csf.nii.gz.nii.gz
Starting download of phantom_1.0mm_msles1_gry... https://brainweb.bic.mni.mcgill.ca/cgi/brainweb2?format_value=raw_short&zip_value=none&who_name=Emil+Ljungberg&who_institution=Lund+Univers